In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def generate_submission(model, test_df, image_dir="test_resized", device="cuda"):
    """Your exact prediction logic as a function"""
    soil_classes = ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']
    le = LabelEncoder()
    le.fit(soil_classes)
    
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    predicted_labels = []
    for img_id in test_df["image_id"]:
        img_path = os.path.join(image_dir, img_id)
        try:
            image = Image.open(img_path).convert("RGB")
            image = test_transform(image).unsqueeze(0).to(device)
            
            with torch.no_grad():
                outputs = model(image)
                _, predicted_class = torch.max(outputs, 1)
                predicted_labels.append(predicted_class.item())
        except Exception as e:
            print(f"Failed to process {img_id}: {e}")
            predicted_labels.append(0)
    
    return pd.DataFrame({
        "image_id": test_df["image_id"],
        "soil_type": le.inverse_transform(predicted_labels)
    })